# Data Pre-processing

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, Dropout, Reshape
from sklearn.metrics import classification_report

In [19]:
weather_raw_df = pd.read_csv('the_ultimate_complete_data_delet.csv',',',header=0,dtype='float64',nrows=100000)
weather_raw_df.drop(['Belfort Rain Gauge','Logger Battery Voltage','Year','Julian Day','Time'],axis=1,inplace=True)

weather_labels = weather_raw_df['Precipitation (Tipping Bucket) 3H'].copy()

weather_raw_df.drop(['Precipitation (Tipping Bucket) 3H'],axis=1,inplace=True)

weather_raw_df.head(10)

,Net Radiometer,Incoming Shortwave Radiation,Reflected Shortwave Radiation,Relative Humidity,Ambient Air Temperature,Barometric Pressure,Wind Direction,Wind Speed,Wind Speed at Rain Gauge,Precipitation (Tipping Bucket)
0,-3.076,-2.500,0.000,61.36,2.046,101.4,107.9,2.639,2.217,9.0
1,-3.080,-2.500,-0.629,61.69,1.972,101.4,109.9,2.082,1.647,9.0
2,-3.085,-2.500,0.000,62.97,1.822,101.4,97.5,1.986,1.491,9.0
3,-3.089,-2.500,0.000,62.50,1.699,101.4,91.7,2.513,1.975,9.0
4,-3.094,-2.500,0.000,63.37,1.737,101.3,92.5,3.000,2.501,9.0
5,-3.097,-2.500,0.000,65.99,1.371,101.3,95.5,2.406,1.848,9.0
6,-3.100,-2.500,0.000,67.20,1.436,101.3,94.9,2.722,2.252,9.0
7,-3.102,-1.875,0.000,65.39,1.531,101.3,92.9,2.956,2.437,9.0
8,-3.104,-2.500,0.000,65.73,1.559,101.3,97.7,3.148,2.680,9.0
9,-3.105,-2.500,0.000,65.12,1.549,101.3,99.9,3.613,3.109,9.0


In [20]:
mask = weather_raw_df['Precipitation (Tipping Bucket)'] > 0
mask2 = weather_raw_df['Precipitation (Tipping Bucket)'] <= 0
weather_raw_df.loc[mask,'Precipitation (Tipping Bucket)'] = 1
weather_raw_df.loc[mask2,'Precipitation (Tipping Bucket)'] = 0

In [21]:
i = 0
for column in weather_raw_df:
    if i < 9:
        weather_raw_df.loc[:,column] += np.absolute(weather_raw_df[column].min())
        weather_raw_df.loc[:,column] /= weather_raw_df[column].max()
        weather_raw_df.loc[:,column] -= np.mean(weather_raw_df[column])
        #weather_raw_df.loc[:,column] /= np.std(weather_raw_df[column])
    i += 1

In [22]:
mask = weather_labels > 0
mask2 = weather_labels <= 0
weather_labels.loc[mask] = 1
weather_labels.loc[mask2] = 0

In [23]:
weather_raw_df.head(10)

,Net Radiometer,Incoming Shortwave Radiation,Reflected Shortwave Radiation,Relative Humidity,Ambient Air Temperature,Barometric Pressure,Wind Direction,Wind Speed,Wind Speed at Rain Gauge,Precipitation (Tipping Bucket)
0,-0.259866,-0.120164,-0.053239,-0.148139,-0.115059,-0.000769,-0.010221,0.061435,0.076520,1.0
1,-0.260260,-0.120164,-0.054044,-0.145177,-0.116331,-0.000769,-0.009949,0.013376,0.012475,1.0
2,-0.260753,-0.120164,-0.053239,-0.133687,-0.118910,-0.000769,-0.011635,0.005093,-0.005053,1.0
3,-0.261148,-0.120164,-0.053239,-0.137906,-0.121024,-0.000769,-0.012424,0.050563,0.049329,1.0
4,-0.261641,-0.120164,-0.053239,-0.130096,-0.120371,-0.001262,-0.012315,0.092582,0.108430,1.0
5,-0.261937,-0.120164,-0.053239,-0.106577,-0.126663,-0.001262,-0.011907,0.041331,0.035059,1.0
6,-0.262233,-0.120164,-0.053239,-0.095716,-0.125546,-0.001262,-0.011989,0.068596,0.080452,1.0
7,-0.262430,-0.119685,-0.053239,-0.111963,-0.123912,-0.001262,-0.012260,0.088786,0.101239,1.0
8,-0.262627,-0.120164,-0.053239,-0.108911,-0.123431,-0.001262,-0.011608,0.105352,0.128542,1.0
9,-0.262726,-0.120164,-0.053239,-0.114387,-0.123603,-0.001262,-0.011309,0.145473,0.176744,1.0


# Train Test Data Split

In [24]:
x_train, x_test, y_train, y_test = train_test_split(weather_raw_df, weather_labels, test_size=0.2, random_state=69, shuffle=True)

# CNN Model

In [29]:
model = Sequential([
     Reshape((1,10), input_shape=(10,)),
     Conv1D(32, (1), activation='relu', input_shape=(1,10)),
     Conv1D(64, (1), activation='relu'),
     MaxPool1D(1),
     Flatten(),
     Dense(128, activation='relu'),
     Dropout(0.2),
     Dense(128, activation='relu'),
     Dropout(0.2),
     Dense(128, activation='relu'),
     Dense(128, activation='relu'),
     Dense(2, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# info = model.fit(x=x_train, y=y_train, validation_data=(x_val,y_val), epochs=15)
info = model.fit(x=x_train, y=y_train, validation_split=0.2, epochs=3)

Train on 64000 samples, validate on 16000 samples
Epoch 1/3
64000/64000 [==============================] - 12s 188us/step - loss: 0.1837 - acc: 0.9553 - val_loss: 0.1529 - val_acc: 0.9556
Epoch 2/3
64000/64000 [==============================] - 11s 179us/step - loss: 0.1462 - acc: 0.9555 - val_loss: 0.1399 - val_acc: 0.9556
Epoch 3/3
64000/64000 [==============================] - 11s 179us/step - loss: 0.1386 - acc: 0.9553 - val_loss: 0.1364 - val_acc: 0.9556


In [27]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions,axis=1)

In [28]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19145
         1.0       0.83      0.01      0.01       855

   micro avg       0.96      0.96      0.96     20000
   macro avg       0.90      0.50      0.49     20000
weighted avg       0.95      0.96      0.94     20000



# SVM Model